In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import datetime

In [ ]:
from google.colab import files
upload = files.upload()

Saving weblog.csv to weblog (1).csv


In [ ]:
dataset = pd.read_csv('weblog.csv')
dataset.head()

,IP,Time,URL,Staus
0,10.128.2.1,[29/Nov/2017:06:58:55,GET /login.php HTTP/1.1,200
1,10.128.2.1,[29/Nov/2017:06:59:02,POST /process.php HTTP/1.1,302
2,10.128.2.1,[29/Nov/2017:06:59:03,GET /home.php HTTP/1.1,200
3,10.131.2.1,[29/Nov/2017:06:59:04,GET /js/vendor/moment.min.js HTTP/1.1,200
4,10.130.2.1,[29/Nov/2017:06:59:06,GET /bootstrap-3.3.7/js/bootstrap.js HTTP/1.1,200


In [ ]:
dataset.iloc[0:1,0]

0    10.128.2.1
Name: IP, dtype: object

In [ ]:
df = dataset.iloc[:580,:]
df.head()

,IP,Time,URL,Staus
0,10.128.2.1,[29/Nov/2017:06:58:55,GET /login.php HTTP/1.1,200
1,10.128.2.1,[29/Nov/2017:06:59:02,POST /process.php HTTP/1.1,302
2,10.128.2.1,[29/Nov/2017:06:59:03,GET /home.php HTTP/1.1,200
3,10.131.2.1,[29/Nov/2017:06:59:04,GET /js/vendor/moment.min.js HTTP/1.1,200
4,10.130.2.1,[29/Nov/2017:06:59:06,GET /bootstrap-3.3.7/js/bootstrap.js HTTP/1.1,200


In [ ]:
data = {'IP_Address':['10.128.2.1'], 'Time_Difference_Mean':[99], 'Time_Difference_Variance':[54], 'Time_Difference_Sum':[105], 'Time_Difference_Maximum':[15], 'Character-bigrams':[45], 'Character-trigrams':[34], 'Character-ngrams':[74], 'Count_of_most_visited_page':[14], 'Status':[200], 'Number_of_records':[40]}
work_dataset = pd.DataFrame(data)
work_dataset

,IP_Address,Time_Difference_Mean,Time_Difference_Variance,Time_Difference_Sum,Time_Difference_Maximum,Character-bigrams,Character-trigrams,Character-ngrams,Count_of_most_visited_page,Status,Number_of_records
0,10.128.2.1,99,54,105,15,45,34,74,14,200,40


In [ ]:
def time_stats(dataframe):
    time_array = dataframe.iloc[:, 1].values
    max_time_diff = 0
    count = 0
    sum_time_diff = 0
    for i in range(1, len(time_array)):
        time1 = time_array[i]
        time2 = time_array[i-1]
        index1 = 0
        for char in time1:
            if(char==':'):
                break
            index1 = index1 +1
        time1 = time1[index1+1:]
        index2 = 0
        for char in time2:
            if(char==':'):
                break
            index2 = index2 + 1
        time2 = time2[index2+1:]
        dtime1 = datetime.datetime.strptime(time1, '%H:%M:%S')
        dtime2 = datetime.datetime.strptime(time2, '%H:%M:%S')
        diff = (dtime1 - dtime2).seconds
        sum_time_diff = sum_time_diff + diff
        count = count + 1
        
        if max_time_diff < diff:
            max_time_diff = diff
    
    mean_time_diff = 0
    if count!=0:
        mean_time_diff = sum_time_diff/count
    
    var_sum = 0
    for i in range(1, len(time_array)):
        time1 = time_array[i]
        time2 = time_array[i-1]
        index1 = 0
        for char in time1:
            if(char==':'):
                break
            index1 = index1 +1
        time1 = time1[index1+1:]
        index2 = 0
        for char in time2:
            if(char==':'):
                break
            index2 = index2 + 1
        time2 = time2[index2+1:]
        dtime1 = datetime.datetime.strptime(time1, '%H:%M:%S')
        dtime2 = datetime.datetime.strptime(time2, '%H:%M:%S')
        diff = (dtime1 - dtime2).seconds
        curvar = (mean_time_diff-diff)*(mean_time_diff-diff)
        var_sum = var_sum + curvar
    
    variance_time_diff = 0
    if count != 0:
        variance_time_diff = var_sum/count
    return max_time_diff, mean_time_diff, sum_time_diff, variance_time_diff

In [ ]:
time_stats(df)

(23466, 95.58376511226253, 55343, 1112702.1497191428)

In [ ]:
import re
import string
import nltk
nltk.download('punkt')
import nltk.corpus
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import bigrams

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def most_frequent(List): 
    return max(set(List), key = List.count)

In [ ]:
def bigram_stats(dataframe):
    url_array = dataframe.iloc[:,2].values
    full_string = ""
    for url in url_array:
        full_string = full_string + url
    full_string = full_string.lower()
    full_string = re.sub(r'\d+', '', full_string)
    table = str.maketrans({key: None for key in string.punctuation})
    full_string = full_string.translate(table) 
    full_string = full_string.strip()
    full_string_tokens = word_tokenize(full_string)
    for char in full_string_tokens:
        if char == 'get':
            full_string_tokens.remove('get')
    for char in full_string_tokens:
        if char == 'httpget':
            full_string_tokens.remove('httpget')
    for char in full_string_tokens:
        if char == 'httppost':
            full_string_tokens.remove('httppost')
    for char in full_string_tokens:
        if char == 'post':
            full_string_tokens.remove('post')
    
    count_most_visited_page = 0
    if len(full_string_tokens)!=0:
        most_visited_page = most_frequent(full_string_tokens)
        count_most_visited_page = full_string_tokens.count(most_visited_page)
        
    
    count_most_appearing_bigram = 0
    bigrams = list(nltk.bigrams(full_string_tokens))
    if len(bigrams)!=0:
        most_appearing_bigram = most_frequent(bigrams)
        count_most_appearing_bigram = bigrams.count(most_appearing_bigram)
    
    count_most_appearing_trigram = 0
    trigrams = list(nltk.trigrams(full_string_tokens))
    if len(trigrams)!=0:
        most_appearing_trigram = most_frequent(trigrams)
        count_most_appearing_trigram = trigrams.count(most_appearing_trigram)
    
    count_most_appearing_ngram = 0
    ngrams = list(nltk.ngrams(full_string_tokens, 6))
    if len(ngrams)!=0:
        most_appearing_ngram = most_frequent(ngrams)
        count_most_appearing_ngram = ngrams.count(most_appearing_ngram)
        
    return count_most_visited_page, count_most_appearing_bigram, count_most_appearing_trigram, count_most_appearing_ngram

In [ ]:
bigram_stats(df)

(79, 27, 17, 5)

In [ ]:
from collections import Counter

In [ ]:
def most_visited_ip(dataframe):
    ip = dataframe.iloc[:,0].values
    x = Counter(ip)
    return x.most_common(1)[0][0]

In [ ]:
most_visited_ip(df)

'10.131.2.1'

In [ ]:
def most_freq_status(dataframe):
    status = dataframe.iloc[:,3].values
    x = Counter(status)
    return x.most_common(1)[0][0]

In [ ]:
most_freq_status(df)

200

In [ ]:
pip install nltk==3.6.2

In [ ]:
startindex = 0
endindex = 0
prevdate = ""
p = 1
for i,row in dataset.iterrows():
    
    date = row['Time'][1:12]
    if i==0:
        prevdate = date
    else:
        if date == prevdate:
            endindex = endindex + 1
        else:
            data = dataset.iloc[startindex:endindex+1, :]
            max_time_diff, mean_time_diff, sum_time_diff, variance_time_diff = time_stats(data)
            #print(max_time_diff, mean_time_diff, sum_time_diff, variance_time_diff)
            count_most_visited_page, count_most_appearing_bigram, count_most_appearing_trigram, count_most_appearing_ngram = bigram_stats(data)
            #print(count_most_visited_page, count_most_appearing_bigram, count_most_appearing_trigram, count_most_appearing_ngram)
            most_vis_ip = most_visited_ip(data)
            most_frequent_status = most_freq_status(data)
            #print(most_vis_ip, most_frequent_status, end = ' ')
            #print()
            if most_vis_ip == None:
                most_vis_ip = '10.130.2.1'
            if most_frequent_status == None:
                most_frequent_status = 200
            work_dataset.loc[p]=[count_most_appearing_bigram, count_most_appearing_ngram, count_most_appearing_trigram, count_most_visited_page, most_vis_ip, endindex - startindex + 1, most_frequent_status, max_time_diff, mean_time_diff, sum_time_diff, variance_time_diff]
            startindex = endindex
            #endindex = endindex + 1
            prevdate = date
            p = p + 1
            #print(max_time_diff, mean_time_diff, sum_time_diff, variance_time_diff)

In [ ]:
work_dataset

,IP_Address,Time_Difference_Mean,Time_Difference_Variance,Time_Difference_Sum,Time_Difference_Maximum,Character-bigrams,Character-trigrams,Character-ngrams,Count_of_most_visited_page,Status,Number_of_records
0,10.128.2.1,99,54,105,15,45,34,74,14.000000,200,4.000000e+01
1,27,5,17,79,10.131.2.1,580,200,23466,95.583765,55343,1.112702e+06
2,69,26,46,343,10.131.0.1,2991,200,86399,143.820401,430023,1.013628e+07
3,13,5,10,44,10.131.0.1,468,200,9569,189.541756,88516,8.867826e+05
4,16,4,10,32,10.130.2.1,168,200,12157,508.694611,84952,2.615859e+06
...,...,...,...,...,...,...,...,...,...,...,...
63,6,1,3,14,10.131.0.1,139,200,40719,799.347826,110310,2.461098e+07
64,5,2,3,11,10.131.0.1,98,200,26367,801.041237,77701,1.140965e+07
65,6,1,3,19,10.128.2.1,83,200,37698,1609.439024,131974,2.751322e+07
66,6,2,4,21,10.130.2.1,123,200,22220,602.032787,73448,8.091683e+06


In [ ]:
Ip_rep = np.array([])
for i,row in work_dataset.iterrows():
    ip = str(row[4])
    ip = ip.replace('.' , '')
    a = int(ip)
    Ip_rep = np.append(Ip_rep, a)
Ip_rep
work_dataset['IP_rep'] = Ip_rep

In [ ]:
work_dataset

,IP_Address,Time_Difference_Mean,Time_Difference_Variance,Time_Difference_Sum,Time_Difference_Maximum,Character-bigrams,Character-trigrams,Character-ngrams,Count_of_most_visited_page,Status,Number_of_records,IP_rep
0,10.128.2.1,99,54,105,15,45,34,74,14.000000,200,4.000000e+01,15.0
1,27,5,17,79,10.131.2.1,580,200,23466,95.583765,55343,1.112702e+06,1013121.0
2,69,26,46,343,10.131.0.1,2991,200,86399,143.820401,430023,1.013628e+07,1013101.0
3,13,5,10,44,10.131.0.1,468,200,9569,189.541756,88516,8.867826e+05,1013101.0
4,16,4,10,32,10.130.2.1,168,200,12157,508.694611,84952,2.615859e+06,1013021.0
...,...,...,...,...,...,...,...,...,...,...,...,...
63,6,1,3,14,10.131.0.1,139,200,40719,799.347826,110310,2.461098e+07,1013101.0
64,5,2,3,11,10.131.0.1,98,200,26367,801.041237,77701,1.140965e+07,1013101.0
65,6,1,3,19,10.128.2.1,83,200,37698,1609.439024,131974,2.751322e+07,1012821.0
66,6,2,4,21,10.130.2.1,123,200,22220,602.032787,73448,8.091683e+06,1013021.0


In [ ]:
work_dataset.drop('IP_Address', axis=1, inplace=True)

In [ ]:
work_dataset

,Time_Difference_Mean,Time_Difference_Variance,Time_Difference_Sum,Time_Difference_Maximum,Character-bigrams,Character-trigrams,Character-ngrams,Count_of_most_visited_page,Status,Number_of_records,IP_rep
0,99,54,105,15,45,34,74,14.000000,200,4.000000e+01,15.0
1,5,17,79,10.131.2.1,580,200,23466,95.583765,55343,1.112702e+06,1013121.0
2,26,46,343,10.131.0.1,2991,200,86399,143.820401,430023,1.013628e+07,1013101.0
3,5,10,44,10.131.0.1,468,200,9569,189.541756,88516,8.867826e+05,1013101.0
4,4,10,32,10.130.2.1,168,200,12157,508.694611,84952,2.615859e+06,1013021.0
...,...,...,...,...,...,...,...,...,...,...,...
63,1,3,14,10.131.0.1,139,200,40719,799.347826,110310,2.461098e+07,1013101.0
64,2,3,11,10.131.0.1,98,200,26367,801.041237,77701,1.140965e+07,1013101.0
65,1,3,19,10.128.2.1,83,200,37698,1609.439024,131974,2.751322e+07,1012821.0
66,2,4,21,10.130.2.1,123,200,22220,602.032787,73448,8.091683e+06,1013021.0


In [ ]:
X = work_dataset.iloc[:,:].values

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X = sc_X.fit_transform(X)
X 


ValueError: ignored